In [14]:
import PIL.Image

In [11]:
import keras

In [43]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'desktop/data/train'
validation_data_dir = 'desktop/data/validation'
nb_train_samples = 160
nb_validation_samples = 40
epochs = 50
batch_size = 10


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * 80 + [1] * 80)

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * int(np.divide(nb_validation_samples , 2)) + [1] *int(np.divide (nb_validation_samples, 2)))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Train on 160 samples, validate on 40 samples
Epoch 1/50
160/160 [==============================] - 1s 6ms/step - loss: 1.6626 - acc: 0.5688 - val_loss: 1.0485 - val_acc: 0.5250
Epoch 2/50
160/160 [==============================] - 1s 4ms/step - loss: 0.7141 - acc: 0.6875 - val_loss: 0.5307 - val_acc: 0.7000
Epoch 3/50
160/160 [==============================] - 1s 4ms/step - loss: 0.3250 - acc: 0.8187 - val_loss: 0.7727 - val_acc: 0.6750
Epoch 4/50
160/160 [==============================] - 1s 4ms/step - loss: 0.5968 - acc: 0.7875 - val_loss: 0.8294 - val_acc: 0.6750
Epoch 5/50
160/160 [==============================] - 1s 4ms/step - loss: 0.3777 - acc: 0.8750 - val_loss: 0.5151 - val_acc: 0.8000
Epoch 6/50
160/160 [==============================] - ETA: 0s - loss: 0.1479 - acc: 0.953 - 1s 5ms/step - loss: 0.1418 - acc: 0.9562 - val_loss: 0.9495 - val_acc: 0.6500
Epoch 7/50
160/160 [========================